# Evaluation per experimental parameter

In [73]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from utils_visualizations import flatten_data, plots_per_condition, filter_data, plot_combined_boxplots, descreptives
import csv

# remove waringns
import warnings
warnings.filterwarnings("ignore")

## Load data for evaluation: Filter for one experimental parameter

In [80]:
# set file path to json files (change model) -> get names of json files
model_name = 'rf'
json_path = "/Users/nadja/Documents/University/Master_Statistik/WS2023_24/SLDS/slds_stratification/Simulations/Final_Setup/results/" + model_name + "/"
json_files = os.listdir(json_path)

# load data from json file
data = []
for file in json_files:
    with open(json_path + file) as f:
        data_new = json.load(f)
        data = data + data_new

# flatten dictionary and convert to dataframe
data_all_flatten, keys_dic = flatten_data(data)
data = pd.DataFrame(data_all_flatten)
print('Shape of data', data.shape)
print('Colum names of data', data.columns)
print('Number of Colums', data.columns.shape)
print('Keys of dictionary (', len(keys_dic), ')', keys_dic)


# check for duplicates ind data
print('Duplicates: ', data.astype(str).duplicated().sum())


data.head()

Shape of data (2880, 75)
Colum names of data Index(['model_info_model', 'model_info_n_train', 'model_info_n_test',
       'model_info_n_features', 'model_info_noise',
       'model_info_transformation', 'model_info_group_size',
       'model_info_n_folds', 'model_info_n_iter', 'model_info_n_repetitions',
       'model_info_scoring', 'model_info_n_jobs', 'model_info_json_file',
       'repetition', 'random_state', 'hyperparameters_same',
       'unstratified_results_train r2', 'unstratified_results_test r2',
       'unstratified_results_train mse', 'unstratified_results_test mse',
       'unstratified_results_train mae', 'unstratified_results_test mae',
       'stratified_results_train r2', 'stratified_results_test r2',
       'stratified_results_train mse', 'stratified_results_test mse',
       'stratified_results_train mae', 'stratified_results_test mae',
       'cv_unstratified_iterations_mean_fit_time',
       'cv_unstratified_iterations_std_fit_time',
       'cv_unstratified_iterat

,model_info_model,model_info_n_train,model_info_n_test,model_info_n_features,model_info_noise,model_info_transformation,model_info_group_size,model_info_n_folds,model_info_n_iter,model_info_n_repetitions,...,cv_folds_descriptives_unstratified_intersection_area,cv_folds_descriptives_stratified_ks_statistic,cv_folds_descriptives_stratified_p_value,cv_folds_descriptives_stratified_intersection_area,unstratified_best_params_min_samples_split,unstratified_best_params_min_samples_leaf,unstratified_best_params_max_features,stratified_best_params_min_samples_split,stratified_best_params_min_samples_leaf,stratified_best_params_max_features
0,rf,200,100000,8,0,identity,5,5,200,20,...,0.866526,0.025,1.0,0.962943,2,1,5,4,1,7
1,rf,200,100000,8,0,identity,5,5,200,20,...,0.851083,0.025,1.0,0.952666,3,1,5,3,1,5
2,rf,200,100000,8,0,identity,5,5,200,20,...,0.892708,0.025,1.0,0.955146,2,1,7,3,1,6
3,rf,200,100000,8,0,identity,5,5,200,20,...,0.858054,0.025,1.0,0.954002,2,1,4,2,1,6
4,rf,200,100000,8,0,identity,5,5,200,20,...,0.847100,0.025,1.0,0.940897,3,1,6,4,1,5


In [81]:
data.shape

(2880, 75)

In [75]:
parameter_grid = {
    "model_info_model": [model_name],
    "model_info_transformation": ['identity', 'log', 'sqrt'],
    "model_info_n_train": [200, 1000],
    "model_info_group_size": [5, 10],
    "model_info_noise": [0, 3]
}
parameter_combinatons = generate_hyperparameter_combinations_dict(parameter_grid)
len(parameter_combinatons)

parameter_combination = parameter_combinatons[0]
parameter_combination

{'model_info_model': 'rf',
 'model_info_transformation': 'identity',
 'model_info_n_train': 200,
 'model_info_group_size': 5,
 'model_info_noise': 0}

In [76]:
# write unique identifier of parameter_combination for saving
parameter_combination_string = ''
for key, value in parameter_combination.items():
    parameter_combination_string = parameter_combination_string + '_' + str(value)
parameter_combination_string = parameter_combination_string[1:]
parameter_combination_string

'rf_identity_200_5_0'

In [77]:
# filter data for experimental parameter combination
filtered_data = data[
    (data['model_info_model'] == parameter_combination['model_info_model']) &
    (data['model_info_transformation'] == parameter_combination['model_info_transformation']) &
    (data['model_info_n_train'] == parameter_combination['model_info_n_train']) &
    (data['model_info_group_size'] == parameter_combination['model_info_group_size']) &
    (data['model_info_noise'] == parameter_combination['model_info_noise'])
]
print('experimental parameter combination:', parameter_combination, 'with shape', filtered_data.shape)


path_evaluation_tables = "/Users/nadja/Documents/University/Master_Statistik/WS2023_24/SLDS/slds_stratification/Simulations/Final_Setup/Evaluation/tables/" 
path_evaluation_plots = "/Users/nadja/Documents/University/Master_Statistik/WS2023_24/SLDS/slds_stratification/Simulations/Final_Setup/Evaluation/plots/" 

if not os.path.exists(path_evaluation_tables + model_name + "/" + parameter_combination_string):
    os.makedirs(path_evaluation_tables + model_name + "/" + parameter_combination_string)
    print('Folder for tables created:', parameter_combination_string)

if not os.path.exists(path_evaluation_plots + model_name + "/" + parameter_combination_string):
    os.makedirs(path_evaluation_plots + model_name + "/" + parameter_combination_string)
    print('Folder for plots created:', parameter_combination_string)



experimental parameter combination: {'model_info_model': 'rf', 'model_info_transformation': 'identity', 'model_info_n_train': 200, 'model_info_group_size': 5, 'model_info_noise': 0} with shape (120, 75)


In [83]:
def save_results_to_csv(file_path, parameter_combination_string, hyp_different, hyp_same, hyp_different_rel):
    header = ['parameter_combination_string', 'hyp_different', 'hyp_same', 'hyp_different_rel']
    data = [parameter_combination_string, hyp_different, hyp_same, hyp_different_rel]

    if not os.path.exists(file_path): # file does not exist
        with open(file_path, 'w', newline='') as f:
            csv_writer = csv.writer(f)
            csv_writer.writerow(header)
        
    with open(file_path, 'a', newline='') as f: # file exists, append row
        csv_writer = csv.writer(f)
        csv_writer.writerow(data)



In [84]:
###  Investigate 'hyperparameters_same'
hyp_same = filtered_data['hyperparameters_same'].value_counts()
hyp_different_rel = hyp_same[0]/(hyp_same[0]+hyp_same[1])
print('hyp_different:', hyp_same[0], '\nhyp_same:', hyp_same[1], '\nrelative difference:', hyp_different_rel)
save_results_to_csv(path_evaluation_tables + 'results_per_parameter.csv', parameter_combination_string, hyp_same[0], hyp_same[1], hyp_different_rel)


hyp_different: 66 
hyp_same: 54 
relative difference: 0.55
